# Install Spark

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://arch

# Start a SparkSession with postgreSQL driver

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Vine_Review").getOrCreate()

# Extract

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles

In [4]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [5]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [6]:
df.count()

6908554

### Create Vine DataFrame

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])

In [8]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|                Y|
|R1QF6RS1PDLU18|          4|            1|          1|   N|                Y|
|R23AICGEDAJQL1|          1|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|     

In [9]:
vine_df.count()

6908554

# Deliverable 2

## Reviews with total_votes >= 20

In [10]:
# Filter total_votes>=20
df_vine_20_total_votes = vine_df.filter("total_votes>=20")

In [11]:
df_vine_20_total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R1RMOCWMV8F4VH|          1|            5|         21|   N|                Y|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|     

In [12]:
df_vine_20_total_votes.count()

96163

## Helpful over total votes ratio

In [13]:
df_vine_20_total_votes=df_vine_20_total_votes.withColumn('helpful_over_total_votes',df_vine_20_total_votes['helpful_votes']/df_vine_20_total_votes['total_votes'])

In [14]:
df_vine_20_total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_over_total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|      0.7741935483870968|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|      0.8571428571428571|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|     0.07407407407407407|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|    0.050505050505050504|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|      0.6338028169014085|
|R1RMOCWMV8F4VH|          1|            5|         21|   N|                Y|     0.23809523809523808|
|R351KVE3VO78W9|          3|           36|         42|   N|              

## Filter helpful over total votes >= 0.50

In [15]:
df_vine_valued_reviews= df_vine_20_total_votes.filter('helpful_over_total_votes>=0.50')

In [16]:
df_vine_valued_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_over_total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|      0.7741935483870968|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|      0.8571428571428571|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|      0.6338028169014085|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|      0.8571428571428571|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|                     1.0|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|      0.7777777777777778|
|R38OLNY5GR1PW6|          5|           44|         44|   N|              

In [17]:
df_vine_valued_reviews.count()

79160

In [18]:
df_paid_reviews=df_vine_valued_reviews.filter('vine=="Y"')

In [19]:
df_paid_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_over_total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R3J2DPYUH6453D|          5|          136|        153|   Y|                N|      0.8888888888888888|
|R25QTFLERWK6SJ|          3|          852|        956|   Y|                N|       0.891213389121339|
|R1BG71K9B61LI1|          4|           39|         39|   Y|                N|                     1.0|
| R8WGX426U56E0|          5|           19|         21|   Y|                N|      0.9047619047619048|
| R614TCIDD36LE|          4|          131|        147|   Y|                N|       0.891156462585034|
|R28X5K1OZBQSBK|          5|           25|         28|   Y|                N|      0.8928571428571429|
|R2CUDWKAUWGHIT|          5|           19|         24|   Y|              

In [20]:
paid_reviews=df_paid_reviews.count()

In [21]:
paid_reviews

1775

In [22]:
df_unpaid_reviews=df_vine_valued_reviews.filter('vine=="N"')

In [23]:
df_unpaid_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_over_total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|      0.7741935483870968|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|      0.8571428571428571|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|      0.6338028169014085|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|      0.8571428571428571|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|                     1.0|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|      0.7777777777777778|
|R38OLNY5GR1PW6|          5|           44|         44|   N|              

In [24]:
unpaid_reviews=df_unpaid_reviews.count()

In [25]:
unpaid_reviews

77385

In [26]:
paid_reviews_5s=df_paid_reviews.filter('star_rating==5').count()

In [27]:
paid_reviews_5s

783

In [28]:
unpaid_reviews_5s=df_unpaid_reviews.filter('star_rating==5').count()

In [29]:
unpaid_reviews_5s

35949

In [30]:
paid_reviews_5s_pct = paid_reviews_5s / paid_reviews
paid_reviews_5s_pct

0.4411267605633803

In [31]:
unpaid_reviews_5s_pct = unpaid_reviews_5s / unpaid_reviews
unpaid_reviews_5s_pct

0.4645473929056019